In [2]:
# ! wget https://raw.githubusercontent.com/callummcdougall/arena-v1/main/w4d1/utils.py

import torch as t
from typing import Union
from torch import nn
import torch.nn.functional as F
import plotly.express as px
import plotly.graph_objects as go
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange
from fancy_einsum import einsum
import os
from tqdm.auto import tqdm
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import wandb
import utils

--2022-11-15 13:03:22--  https://raw.githubusercontent.com/callummcdougall/arena-v1/main/w4d1/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9325 (9.1K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   9.11K  --.-KB/s    in 0.08s   

2022-11-15 13:03:24 (111 KB/s) - ‘utils.py’ saved [9325/9325]



## transposed convolutions

Question - what do you think the formula is relating input_size, kernel_size and output_size in the case of 1D convolutions (with no padding or stride)?

output_size = input_size + kernel_size - 1

since it reverses the shape change of a normal convolution

output_size = input_size - kernel_size + 1

Question - how can this operation be cast as a convolution? In other words, exactly what arrays input and kernel would produce the same output as the transposed convolution above, if we performed a standard convolution on them?

it is necessary to pad the input with zeros kernel_size -1,
the kernel needs to be reversed

In [33]:
def conv_transpose1d_minimal(x: t.Tensor, weights: t.Tensor) -> t.Tensor:
    '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

    x: shape (batch, in_channels, width)
    weights: shape (in_channels, out_channels, kernel_width)

    Returns: shape (batch, out_channels, output_width)
    '''
    # reverse kernel
    weight_mod = t.flip(weights, dims=(2,))
    weight_mod = rearrange(weight_mod, "a b c -> b a c")
    x_width = x.shape[2]
    # change input with padding
    kernel_width = weights.shape[2]
    x_mod = t.zeros((x.shape[0], x.shape[1],x.shape[2]+2*kernel_width-2), dtype=x.dtype)
    x_mod[..., kernel_width-1:kernel_width -1 + x_width] = x
    # apply conv1d, todo: maybe use own implementation
    result = t.nn.functional.conv1d(x_mod, weight_mod)
    return result

utils.test_conv_transpose1d_minimal(conv_transpose1d_minimal)

All tests in `test_conv1d_minimal` passed!


In [39]:
def fractional_stride_1d(x, stride: int = 1):
    '''Returns a version of x suitable for transposed convolutions, i.e. "spaced out" with zeros between its values.
    This spacing only happens along the last dimension.

    x: shape (batch, in_channels, width)

    Example: 
        x = [[[1, 2, 3], [4, 5, 6]]]
        stride = 2
        output = [[[1, 0, 2, 0, 3], [4, 0, 5, 0, 6]]]
    '''
    x_mod = t.zeros((x.shape[0], x.shape[1],(x.shape[2]-1)*stride + 1), dtype=x.dtype)
    x_mod[..., ::stride] = x
    return x_mod

utils.test_fractional_stride_1d(fractional_stride_1d)

def conv_transpose1d(x, weights, stride: int = 1, padding: int = 0) -> t.Tensor:
    '''Like torch's conv_transpose1d using bias=False and all other keyword arguments left at their default values.

    x: shape (batch, in_channels, width)
    weights: shape (out_channels, in_channels, kernel_width)

    Returns: shape (batch, out_channels, output_width)
    '''
    # reverse kernel
    weight_mod = t.flip(weights, dims=(2,))
    weight_mod = rearrange(weight_mod, "a b c -> b a c")
    x = fractional_stride_1d(x,stride=stride)
    x_width = x.shape[2]
    # change input with padding
    kernel_width = weights.shape[2]
    x_mod = t.zeros((x.shape[0], x.shape[1],x.shape[2]+2*kernel_width-2), dtype=x.dtype)
    x_mod[..., kernel_width-1:kernel_width -1 + x_width] = x
    # apply conv1d, todo: maybe use own implementation
    result = t.nn.functional.conv1d(x_mod, weight_mod)
    return result

utils.test_conv_transpose1d(conv_transpose1d)

All tests in `test_fractional_stride_1d` passed!


AssertionError: The values for attribute 'shape' do not match: torch.Size([2, 4, 187]) != torch.Size([2, 4, 185]).